In [66]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

names = []
rating = []
cuisine = []
location = []
region = []
cost_per_person = []
discount = []

for i in range(1, 101):
    url = f'https://www.eazydiner.com/bengaluru/restaurants?page={i}'
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    cards = soup.find_all('div', class_='listing_touch_box__00o7f')

    for card in cards:
        name_tag = card.find_all('a',class_='ellipsis listing_res_name__uVIN8')
        names.append(name_tag[0].get_text(strip=True) if name_tag else np.nan)

        card_text = card.get_text(" ", strip=True)
        m = re.search(r'(\d+\.\d+)', card_text)
        rating.append(m.group(1) if m else np.nan)

        cuisine_tag = card.find_all('div', class_='listing_multi_cuisines__zLLXK')
        cuisine.append(cuisine_tag[0].get_text(" ", strip=True) if cuisine_tag else np.nan)
        
        d=card.find_all('div', class_='ellipsis')
        location.append(re.findall('(^\w.+),',d[0].text) if d else np.nan)
        region.append(re.findall(',\s(\w+\s\w+)',d[0].text) if d else 'Bengaluru')

        m_cost = re.search('₹\s*([\d,]+)', card_text)
        cost_per_person.append(m_cost.group(1) if m_cost else np.nan)
        
        m_disc = re.search('(\d{1,3})\s*%', card_text)
        discount.append(m_disc.group(1) if m_disc else np.nan)

In [68]:
x = pd.DataFrame({
    "name": names,
    "rating": rating,
    "cuisine": cuisine,
    "location": location,
    "region": region,
    "cost_per_person": cost_per_person,
    "discount": discount
})

x.head()

,name,rating,cuisine,location,region,cost_per_person,discount
0,Chin Lung Resto Bar,4.6,Multicuisine,[Residency Road],[Central Bengaluru],2000,25
1,Chin Lung Resto Bar,4.6,Multicuisine,[Residency Road],[Central Bengaluru],2000,25
2,"Atlantis - Brewpub, Cocktails & Kitchen",4.2,Multicuisine,[HSR],[South Bengaluru],2000,25
3,"Atlantis - Brewpub, Cocktails & Kitchen",4.2,Multicuisine,[HSR],[South Bengaluru],2000,25
4,Chin Lung Brewery - Indiranagar,4.6,Multicuisine,[Indiranagar],[East Bengaluru],2000,25


In [69]:
x.shape

(1600, 7)

In [70]:
x.to_csv('Eazy_Dinner.csv')

# Exploratory Data Analysis using Pandas

In [88]:
import pandas as pd
import numpy as np
r=pd.read_csv('Eazy_Dinner.csv')
r

,Unnamed: 0,name,rating,cuisine,location,region,cost_per_person,discount
0,0,Chin Lung Resto Bar,4.6,Multicuisine,['Residency Road'],['Central Bengaluru'],2000,25
1,1,Chin Lung Resto Bar,4.6,Multicuisine,['Residency Road'],['Central Bengaluru'],2000,25
2,2,"Atlantis - Brewpub, Cocktails & Kitchen",4.2,Multicuisine,['HSR'],['South Bengaluru'],2000,25
3,3,"Atlantis - Brewpub, Cocktails & Kitchen",4.2,Multicuisine,['HSR'],['South Bengaluru'],2000,25
4,4,Chin Lung Brewery - Indiranagar,4.6,Multicuisine,['Indiranagar'],['East Bengaluru'],2000,25
...,...,...,...,...,...,...,...,...
1595,1595,Hunan Square,4.9,Pan Asian,['Electronic City'],['South Bengaluru'],2000,25
1596,1596,Secrets Of Ocean,4.0,Multicuisine,['Chikkabanavara'],['North Bengaluru'],2000,25
1597,1597,Secrets Of Ocean,4.0,Multicuisine,['Chikkabanavara'],['North Bengaluru'],2000,25
1598,1598,Sidewalk Cafe & Bar,4.3,Multicuisine,['The Paul'],[],2000,10


In [89]:
r.tail()

,Unnamed: 0,name,rating,cuisine,location,region,cost_per_person,discount
1595,1595,Hunan Square,4.9,Pan Asian,['Electronic City'],['South Bengaluru'],2000,25
1596,1596,Secrets Of Ocean,4.0,Multicuisine,['Chikkabanavara'],['North Bengaluru'],2000,25
1597,1597,Secrets Of Ocean,4.0,Multicuisine,['Chikkabanavara'],['North Bengaluru'],2000,25
1598,1598,Sidewalk Cafe & Bar,4.3,Multicuisine,['The Paul'],[],2000,10
1599,1599,Sidewalk Cafe & Bar,4.3,Multicuisine,['The Paul'],[],2000,10


# Removed unwanted coloumns

In [90]:
r.drop('Unnamed: 0', axis=1, inplace=True)


# Data types

In [101]:
r.dtypes

name                object
rating             float64
cuisine             object
location            object
region              object
cost_per_person      int64
discount             int64
dtype: object

In [102]:
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             1600 non-null   object 
 1   rating           1600 non-null   float64
 2   cuisine          1600 non-null   object 
 3   location         1600 non-null   object 
 4   region           1600 non-null   object 
 5   cost_per_person  1600 non-null   int64  
 6   discount         1600 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 87.6+ KB
